Learning outcomes

* Explain the effect of applying gates to mixed states.
* Determine the measurement probabilities and post-measurement states when measuring a mixed state.
* Calculate expectation values for observables on a given mixed state.

In [1]:
import pennylane as qml
import pennylane.numpy as np

**Cordercise N.2.1a** The code below intends to calculate the effect of a quantum gate U on a quantum state with density operator rho. Complete the missing lines to compute the resulting density operator after applying U on rho.

In [4]:
def apply_gate_mixed(rho,U):

    """
    Args:
        rho: (np.array(array[complex]): The density matrix of the input state
        U (np.array(array[complex])): A matrix representing the unitary gate U to be applied.

    Returns:
        (np.array([array[complex]])): The density matrix of the output state.
    """
    U_adjoint = np.transpose(np.conjugate(U))
    return np.dot(U,np.dot(rho,U_adjoint))

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_mixed(rho,U).round(2))


A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[[0.62+0.j   0.25+0.22j]
 [0.25-0.22j 0.38+0.j  ]]


Cordercise N.2.1b Use PennyLane to complete the circuit apply_gate_circuit below, which does the following:

* Prepares a mixed qubit rho. To this end, you can use qml.QubitDensityMatrix.
* Applies an arbitrary unitary. You can still use qml.QubitUnitary when working with mixed qubits.
* Outputs the density matrix of the resulting quantum state. Since we are use the default.mixed device, qml.state() will do this for us.

In [5]:
dev = qml.device("default.mixed", wires=1)

@qml.qnode(dev)
def apply_gate_circuit(rho,U):

    """
    Args:
        rho: (np.array(array[complex]): The density matrix of the input state.
        U (np.array(array[complex])): A matrix representing the unitary gate U to be applied.

    Returns:
        (np.array([array[complex]])): The density matrix of the output state.
    """

    # Prepare a state with density operator rho
    qml.QubitDensityMatrix(rho,wires=0)
    # Apply the unitary U
    qml.QubitUnitary(U, wires=0)

    return qml.state()

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_circuit(rho,U).round(2))


A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[[0.62+0.j   0.25+0.22j]
 [0.25-0.22j 0.38+0.j  ]]


**Cordercise N.2.2a** As we can see in formula $p(k) = tr(\rho \pi_k)$, some key ingredients in calculating the probability of getting an eigenvalue  for an observable  are the eigenprojectors . Fill in the eigenprojectors function below which, given an observable obs, calculates its eigenprojectors.

In [11]:
def eigenprojectors(obs):

    """
    Args:
        obs (np.array([array[complex]])): A Hermitian operator representing a quantum observable.

    Returns:
        (np.array(array[array[complex]])): An array containing the eigenprojectors of the observable.
        Therefore, it is an array that contains matrices.
    """

    eigenvalues,eigenvectors = np.linalg.eigh(obs)
    projectors = [np.outer(v,np.conj(v).T) for v in eigenvectors]

    return projectors

In [12]:
U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])
obs = np.array([[1/2,0],[0,-1/2]])

print(eigenprojectors(obs))

A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[-0.5  0.5]
--------
[[0. 1.]
 [1. 0.]]
--------
[tensor([[0., 0.],
        [0., 1.]], requires_grad=True), tensor([[1., 0.],
        [0., 0.]], requires_grad=True)]


In [14]:
def eigenprojectors(obs):

    """
    Args:
        obs (np.array([array[complex]])): A Hermitian operator representing a quantum observable.

    Returns:
        (np.array(array[array[complex]])): An array containing the eigenprojectors of the observable.
        Therefore, it is an array that contains matrices.
    """

    eigenvalues,eigenvectors = np.linalg.eigh(obs)
    projectors = [np.outer(v, v.conj()) for v in eigenvectors.T]

    return projectors
U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])
obs = np.array([[1/2,0],[0,-1/2]])

print(eigenprojectors(obs))

[tensor([[0., 0.],
        [0., 1.]], requires_grad=True), tensor([[1., 0.],
        [0., 0.]], requires_grad=True)]


**Cordercise N.2.2b** Now it's time to calculate the probabilities. Given a density operator rho and an observable B, complete the outcome_probs function below to return a list probabilities for each eigenprojector. The eigenprojectors function that you defined in Codercise N.2.2a is available for you to use

In [20]:
def outcome_probs(rho, B):

    """
    Args:
        rho (np.array([array[complex]])): The density matrix of the state before measurement
        B (np.array([array[complex]])): Matrix representation of the measured observable
    Returns:
        (np.array(float)): List of measurement probabilities, in no particular order.
    """

    eigen_projectors = eigenprojectors(B)

    prob_list = np.array([np.trace(rho @ p) for p in eigen_projectors])# Use list comprehension and the relevant formula to find the probabilities

    return prob_list

rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliY(0))

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(B))
print("Measurement probabilities {}".format(outcome_probs(rho,B).round(2)))



State: [[3/4,0],[0,1/4]], Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
Measurement probabilities [0.5+0.j 0.5+0.j]


**Codercise N.2.2c** Now that we've got the gist of how the calculation of outcome probabilities works for mixed states, let's simplify our lives by using PennyLane. The circuit mixed_probs_circuit below is intended to carry out the following operations.

* Prepare a quantum state with density matrix rho.
* Return the probabilities of the measurement outcomes of some observable obs.
Note that PennyLane's qml.probs has an optional argument op, which allows you to specify the observable whose probabilities you want to calculate.

In [2]:
dev = qml.device('default.mixed', wires = 1)

@qml.qnode(dev)
def mixed_probs_circuit(rho, obs):
    """
    Prepares a state with density matrix rho and returns measurement
    probabilities associated to the observable obs.

    Args:
        rho (np.array[array[complex]]): The density matrix we need to prepare
        obs: (pennylane.operation): A PennyLane hermitian operator

    Returns:
        np.array(float): The measurement probabilities as required
    """

    qml.QubitDensityMatrix(rho,wires=0)

    return qml.probs(op=obs)# Return the probabilities associated to obs

rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliY(0)

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(qml.matrix(B)))
print("Measurement probabilities {}".format(mixed_probs_circuit(rho,B)))



State: [[3/4,0],[0,1/4]], Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
Measurement probabilities [0.5 0.5]


In [3]:
from qiskit.visualization import array_to_latex
import qiskit.quantum_info as qi

In [4]:
rho_matrix = qi.DensityMatrix(rho)
rho_matrix.draw('latex', prefix='\\rho = ')

<IPython.core.display.Latex object>

In [5]:
print("Observable: {}".format(qml.matrix(B)))
Y = qi.Operator(qml.matrix(B))
array_to_latex(Y.data, prefix='Y =')

Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]


<IPython.core.display.Latex object>

**Cordercise N.2.3a** Given a density operator rho and an observable B, write a function mixed_expval that calculates the expectation value $\Braket{B}$ using numpy's linear algebra capabilities.


In [6]:
def mixed_expval(rho, B):

    """
    Returns the expectation value of the observable B on state rho

    Args:
        rho (np.array(array(float)): A numpy matrix representing a density matrix
        B (np.array(array(complex))): A numpy matrix representing an observable

    Returns:
        float: The expectation value as required
    """
    matrix = np.dot(rho,B)

    return np.trace(matrix)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliZ(0))

print("State: {}".format(rho))
print("Observable: {}".format(B))
print("Expectation value: {}".format(mixed_expval(rho,B)))


State: [[0.75 0.  ]
 [0.   0.25]]
Observable: [[ 1  0]
 [ 0 -1]]
Expectation value: 0.5


**Cordercise N.2.3b** Now let's do the same using PennyLane! Given a state rho and an observable B, build a quantum circuit `expval_circuit` that prepares the state $\rho$ and returns the expectation value of B. You will need `qml.expval` to do this.

In [ ]:
dev = qml.device('default.mixed', wires = 1)

@qml.qnode(dev)
def expval_circuit(rho,obs):

    """
    Returns the expectation value of the observable obs on state rho

    Args:
        rho (np.array(array(float)): A numpy matrix representing a density matrix
        B (pennylane.operation): A pennylane observable

    Returns:
        np.tensor: A numpy tensor with the expectation value as required
    """
    qml.QubitDensityMatrix(rho,wires=[0])

    return qml.expval(B)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliZ(0)

print("State: {}".format(rho))
print("Observable: {}".format(qml.matrix(B)))
print("Expectation value: {}".format(expval_circuit(rho,B)))


